In [20]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
# sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions
from Bio.PDB.Polypeptide import one_to_three
from Bio.PDB.Polypeptide import three_to_one
from Bio.PDB.PDBParser import PDBParser
from pyCodeLib import *
from small_script.myFunctions import *
from collections import defaultdict
%matplotlib inline
# plt.rcParams['figure.figsize'] = (10,6.180)    #golden ratio
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
plt.rcParams['figure.figsize'] = np.array([16.18033, 10])    #golden ratio
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['figure.dpi'] = 100
plt.rcParams.update({'font.size': 22})

In [41]:

simulationType = "cornichon"
run_n = 5
folder_list = ["positive_inside/submode1_on_off"]
all_data = []

for folder in folder_list:
    for i in range(run_n):
            pre = f"/Users/weilu/Research/server/may_week2_2020//{simulationType}/{folder}/{i}"
            info_file = "info.dat"
            location = f"{pre}/{info_file}"
            try:
                tmp = pd.read_csv(location, sep="\s+")
                tmp = tmp.assign(Run=i, Folder=folder)
                all_data.append(tmp)
            except:
                print(i, folder, location)
                pass
data = pd.concat(all_data)
today = datetime.today().strftime('%m-%d')
outFile = f"/Users/weilu/Research/data/openMM/{simulationType}_{folder.replace('/', '_')}_{today}.csv"
data.reset_index(drop=True).to_csv(outFile)
print(outFile)

/Users/weilu/Research/data/openMM/cornichon_positive_inside_submode1_on_off_05-11.csv


In [42]:
data = pd.read_csv("/Users/weilu/Research/data/openMM/cornichon_positive_inside_submode1_on_off_05-11.csv")


In [43]:
selected = data.sort_values("Q").tail()
selected

,Unnamed: 0,Steps,Q,Qa0.2,Lig_bond,Lig_excl,Lig_inter,Rg,p_inside,Backbone,Rama,Contact,Fragment,Membrane,S_chain,Excl,Excl_CBD,Beta,Pap,Helical,Helix_orientation,Total,Run,Folder
1770,1770,1770,0.42,0.66,0.0,0.0,0.0,20.69,0.90,504.96,-990.63,-303.59,-578.14,-91.15,118.74,40.15,41.80,0.0,0.0,-227.15,47.41,-1479.42,0,positive_inside/submode1_on_off
1775,1775,1775,0.42,0.65,0.0,0.0,0.0,20.92,2.54,496.21,-979.17,-292.40,-561.70,-92.21,109.73,33.98,30.34,0.0,0.0,-215.98,52.15,-1449.39,0,positive_inside/submode1_on_off
1573,1573,1573,0.43,0.69,0.0,0.0,0.0,20.33,1.05,563.98,-951.90,-317.33,-564.52,-92.33,116.72,59.84,46.20,0.0,0.0,-223.93,52.52,-1356.94,0,positive_inside/submode1_on_off
1779,1779,1779,0.43,0.67,0.0,0.0,0.0,20.57,0.52,484.06,-996.39,-294.91,-571.86,-90.87,164.45,52.09,35.54,0.0,0.0,-216.39,31.06,-1438.75,0,positive_inside/submode1_on_off
1752,1752,1752,0.44,0.69,0.0,0.0,0.0,20.25,0.44,508.57,-956.98,-288.99,-566.69,-88.73,88.28,46.22,28.71,0.0,0.0,-223.42,33.35,-1448.38,0,positive_inside/submode1_on_off


In [44]:
selected = data.query("Steps > 2000").sort_values("Q").tail()
selected

,Unnamed: 0,Steps,Q,Qa0.2,Lig_bond,Lig_excl,Lig_inter,Rg,p_inside,Backbone,Rama,Contact,Fragment,Membrane,S_chain,Excl,Excl_CBD,Beta,Pap,Helical,Helix_orientation,Total,Run,Folder
15818,15818,2887,0.41,0.65,0.0,0.0,0.0,21.38,2.02,472.35,-978.10,-351.08,-587.76,-87.50,83.38,34.28,26.82,0.0,0.0,-241.57,21.67,-1634.32,3,positive_inside/submode1_on_off
15406,15406,2475,0.41,0.65,0.0,0.0,0.0,20.96,0.65,478.68,-975.05,-370.35,-574.29,-99.98,98.28,36.92,32.25,0.0,0.0,-218.70,43.47,-1581.03,3,positive_inside/submode1_on_off
15833,15833,2902,0.41,0.63,0.0,0.0,0.0,20.83,2.05,492.76,-969.33,-364.74,-581.89,-87.34,84.22,37.20,37.23,0.0,0.0,-226.70,23.43,-1592.38,3,positive_inside/submode1_on_off
15578,15578,2647,0.42,0.65,0.0,0.0,0.0,20.90,0.60,509.80,-972.48,-350.04,-565.03,-93.84,82.63,28.45,34.21,0.0,0.0,-220.75,28.40,-1552.87,3,positive_inside/submode1_on_off
15408,15408,2477,0.42,0.66,0.0,0.0,0.0,21.17,0.66,481.57,-945.70,-364.43,-576.07,-96.52,75.35,47.67,42.53,0.0,0.0,-218.82,34.08,-1562.87,3,positive_inside/submode1_on_off


In [45]:
selected = data.query("Steps > 2000").sort_values("Qa0.2").tail()
selected

,Unnamed: 0,Steps,Q,Qa0.2,Lig_bond,Lig_excl,Lig_inter,Rg,p_inside,Backbone,Rama,Contact,Fragment,Membrane,S_chain,Excl,Excl_CBD,Beta,Pap,Helical,Helix_orientation,Total,Run,Folder
15003,15003,2072,0.39,0.65,0.0,0.0,0.0,20.58,0.46,498.58,-980.07,-358.09,-572.22,-93.82,112.41,38.40,42.93,0.0,0.0,-224.02,35.19,-1543.64,3,positive_inside/submode1_on_off
15818,15818,2887,0.41,0.65,0.0,0.0,0.0,21.38,2.02,472.35,-978.10,-351.08,-587.76,-87.50,83.38,34.28,26.82,0.0,0.0,-241.57,21.67,-1634.32,3,positive_inside/submode1_on_off
15406,15406,2475,0.41,0.65,0.0,0.0,0.0,20.96,0.65,478.68,-975.05,-370.35,-574.29,-99.98,98.28,36.92,32.25,0.0,0.0,-218.70,43.47,-1581.03,3,positive_inside/submode1_on_off
15408,15408,2477,0.42,0.66,0.0,0.0,0.0,21.17,0.66,481.57,-945.70,-364.43,-576.07,-96.52,75.35,47.67,42.53,0.0,0.0,-218.82,34.08,-1562.87,3,positive_inside/submode1_on_off
14952,14952,2021,0.41,0.66,0.0,0.0,0.0,20.45,0.72,520.65,-950.03,-371.76,-576.03,-95.34,95.77,48.30,42.34,0.0,0.0,-233.32,40.01,-1521.74,3,positive_inside/submode1_on_off


In [46]:
selected = data.sort_values("Qa0.2").tail()
selected

,Unnamed: 0,Steps,Q,Qa0.2,Lig_bond,Lig_excl,Lig_inter,Rg,p_inside,Backbone,Rama,Contact,Fragment,Membrane,S_chain,Excl,Excl_CBD,Beta,Pap,Helical,Helix_orientation,Total,Run,Folder
1771,1771,1771,0.41,0.68,0.0,0.0,0.0,21.18,1.73,490.15,-962.92,-298.65,-569.24,-92.13,162.77,34.15,45.21,0.0,0.0,-224.01,50.51,-1409.38,0,positive_inside/submode1_on_off
1573,1573,1573,0.43,0.69,0.0,0.0,0.0,20.33,1.05,563.98,-951.90,-317.33,-564.52,-92.33,116.72,59.84,46.20,0.0,0.0,-223.93,52.52,-1356.94,0,positive_inside/submode1_on_off
1752,1752,1752,0.44,0.69,0.0,0.0,0.0,20.25,0.44,508.57,-956.98,-288.99,-566.69,-88.73,88.28,46.22,28.71,0.0,0.0,-223.42,33.35,-1448.38,0,positive_inside/submode1_on_off
1765,1765,1765,0.42,0.69,0.0,0.0,0.0,20.21,1.04,528.20,-974.96,-329.92,-570.84,-93.62,142.42,54.17,41.17,0.0,0.0,-216.85,43.82,-1417.57,0,positive_inside/submode1_on_off
8517,8517,586,0.41,0.70,0.0,0.0,0.0,20.10,0.00,581.50,-940.43,-273.45,-547.24,-81.96,177.71,48.44,35.04,0.0,0.0,-200.16,30.45,-1205.15,2,positive_inside/submode1_on_off


In [17]:
pd.set_option('display.max_columns', 500)

In [34]:
pre = "/Users/weilu/Research/server/may_week2_2020/cornichon/results"
os.system(f"mkdir -p {pre}")
i = 1
run = 3
outLocation = f"{pre}/frame_{frame}_run_{run}.pdb"
movieLocation = f"{pre}/../positive_inside/submode1_on_off/{run}/movie.pdb"
getFrame(frame, outLocation, movieLocation=movieLocation)

946


In [40]:
pre = "/Users/weilu/Research/server/may_week2_2020/cornichon/results"
for i, line in selected.iterrows():
    frame = line["Steps"]
    run = line["Run"]
    print(frame, run)
    outLocation = f"{pre}/frame_{frame}_run_{run}.pdb"
    movieLocation = f"{pre}/../positive_inside/submode1_on_off/{run}/movie.pdb"
    getFrame(frame, outLocation, movieLocation=movieLocation)

1771 0
946
1765 0
946
1752 0
946
1573 0
946
586 2
946
